### Řešení čtyřsměrek pomocí optical character recognition (OCR)

- Hlavním cílem tohoto projektu bude *vyhledání daného slova* v obrázku čtyřsměrky a jeho následné *vyznačení*.
- OCR bude zajištěn knihovnou **pytesseract**.
- Pro manipulaci s obrázkem bude použita knihovna **OpenCV**.
- Samotné vyhledávání bude uskutečněno pomocí *regulárních výrazů*.

![alt text](resources/osmismerka.jpg)

In [25]:
import pytesseract
import cv2
import regex as re

- `row_length` udává počet písmen v řádku. Je nutné ho nastavit pro správný běh programu.
- Počet řádků už bude dopočítán.

In [26]:
row_length = 8

### Preprocessing

- Po načtení obrázku a vytvoření jeho kopie je nutné extrahovat jednotlivá písmena.
- Pro maximální úspěšnost tohoto korku se originální obrázek předpřipraví:
1. Převede se do černobílé
2. Aplikuje se rozostření
3. Provede se práhování
4. Připraví se matice pro následující krok
5. Uskuteční se dilatace, což roztáhne plochy, aby se uzavřely malé dírky

In [27]:
img = cv2.imread('resources/osmismerka.jpg')
baseImg = img.copy()

# Preprocessing
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7, 7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
dilated = cv2.dilate(thresh, kernel, iterations=1)

### Extrakce a uspořádání písmen

- Najdou se kontury, které ohraničují jednotlivá písmena.
- Tyto kontury so následně uspořádají tak, aby bylo možné vytvořit dvojrozměrné pole reprezentující čtyřsměrku.
- Nakonec se samotná písmena vyřežou a uloží ve 1D poli `letters`, kde jdou po řádcích jeden za druhým.

In [ ]:
# Find contours
cnts = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# Group contours into rows
def group_contours_by_rows(contours, row_threshold=10):
    rows = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        added = False
        for row in rows:
            # Check if the contour belongs to an existing row
            if abs(row[0][1] - y) < row_threshold:
                row.append((x, y, w, h, c))
                added = True
                break
        if not added:
            # Create a new row if no existing row matches
            rows.append([(x, y, w, h, c)])
    return rows

# Group contours into rows and sort them
rows = group_contours_by_rows(cnts)
rows = sorted(rows, key=lambda row: row[0][1])

# Sort contours in each row by x-coordinate
sorted_contours = []
for row in rows:
    row = sorted(row, key=lambda item: item[0])
    sorted_contours.extend([item[4] for item in row])

letters = []

# Extract letters
for c in sorted_contours:
    x, y, w, h = cv2.boundingRect(c)
    letters.append(img[y-3:y+3 + h, x-3:x+3 + w])

# Display letters
# for i, letter in enumerate(letters):
#     cv2.imshow('Letter', letter)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

- Funkce, která úsečkou vyznačí výsledek.

In [29]:
# Highlight the find in the original image
def draw_line (baseImg, contour1, contour2):
    x1, y1, w1, h1 = cv2.boundingRect(contour1)
    x2, y2, w2, h2 = cv2.boundingRect(contour2)
    cv2.line(baseImg, (int(x1 + 0.5 * w1), int(y1 + 0.5 * h1)), (int(x2 + 0.5 * w2), int(y2 + 0.5 * h2)), (0, 0, 255), 3, lineType=cv2.LINE_AA)
    cv2.imshow('Letter', baseImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### OCR

- Zde se již provádí samotné "čtení" znaků.
- Důležitá je konfigurace `--psm 10`, která dá vědět, že se jedná a samostatná písmena.
- Následně se opraví některé klasické chyby v rozpoznání.
- Výsledek se pak uloží do 2D pole stringů.

In [30]:
row_number = round(len(letters) / row_length)
letterArray = [[0 for x in range(row_length)] for y in range(row_number)]
row = 0

for i, letter in enumerate(letters):
    if i % row_length == 0 and i != 0:
        row += 1
    add = pytesseract.image_to_string(letter, lang='eng', config='--psm 10').strip()
    if '|' in add:
        add = 'I'
    elif '(' in add:
        add = 'C'
    elif len(add) > 1:
        add = add[0].upper()
    else:
        add = add.upper()
    letterArray[row][i % row_length] = add

- Funkce na transponování matice.

In [31]:
def transpose_array(array):
    return [list(row) for row in zip(*array)]

- Funkce na vypočítání indexu v poli `sorted_contours` podle souřadnic písmene ve 2D poli.
- Tento index je důležity pro zakreslení výsledku.

In [32]:
def get_index_of_letter(coordinates):
    return coordinates[0] + coordinates[1] * row_length

### Hledání slova

- Tato funkce vrací index prvního a posledního písmene hledaného slova.
- Jako vstup bere 2D pole písmen, hledané slova a zdali je matice (pole) transponované.
- V normální konfiguraci funguje pro hledání horizontálně v obou směrech.
- Když je `transposed == True`, tak tato funkce funguje i pro vyhledávání vertikálně oběma směry.
- Mezi dvěma módy se liší pouze algoritmus pro vypočítání souřadnic písmena.

In [ ]:
# Returns the indexes of the first and last letter of the word in the array
# Can be used for searching horizontally and vertically when the array is transposed
def array_search(array, word, transposed):
    regex = re.compile(word)
    for i in range(row_number):
        concat = ''
        
        # Create a string from row
        for j in range(row_length):
            concat += array[i][j]
        
        # Search forward
        match = re.search(regex, concat)
        if match:
            if transposed:
                return (get_index_of_letter((i, match.start())), get_index_of_letter((i, match.end() - 1)))
            return (get_index_of_letter((match.start(), i)), get_index_of_letter((match.end() - 1, i)))
        
        # Search backward
        concat = concat[::-1]
        match = re.search(regex, concat)
        if match:
            if transposed:
                return (get_index_of_letter((i, row_length - match.end())), get_index_of_letter((i, row_length - match.start() - 1)))
            return (get_index_of_letter((row_length - match.end(), i)), get_index_of_letter((row_length - match.start() - 1, i)))

### Finále

-Tato funkce zaštiťuje hledání všemi čtyřmi směry a následné vyznačení výsledku.

In [ ]:
def search(word):
    word = word.upper()
    
    # Search horizontally
    indexes = array_search(letterArray, word, False)
    
    # Search vertically
    if indexes is None:
        letterArrayT = transpose_array(letterArray)
        indexes = array_search(letterArrayT, word, True)
    
    # Highlight result
    if indexes is not None:
        draw_line(baseImg.copy(), sorted_contours[indexes[0]], sorted_contours[indexes[1]])
    else:
        print("Word not found")

In [35]:
search("ROBOT")